In [3]:
import numpy as np  
#import matplotlib.pyplot as plt
np.random.seed(0)
a = np.random.randint(1, 10, size=(3, 3))
print(a)

[[6 1 4]
 [4 8 4]
 [6 3 5]]


In [12]:
print('hi')

def test(x,y,z):
    if x < z:
        return x, y
    return z
import torch
    
class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # Define the layers of the model
        layers = [
        torch.nn.Linear(10, 20),
        torch.nn.ReLU(),
        torch.nn.Dropout(0.2),
        torch.nn.Linear(20, 5),
        torch.nn.ReLU()
    ]
        self.hidden_layers = torch.nn.Sequential(*layers)
        
class My2Model(torch.nn.Module):
    def __init__(self):
        super(My2Model, self).__init__()
        # Define the layers of the model
        
        self.fc1 = torch.nn.Linear(10, 20)
        self.relu1 = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(0.2)
        self.fc2 = torch.nn.Linear(20, 5)
        self.relu2 = torch.nn.ReLU()

        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu2(x)
        return x

hi


In [13]:
print(test(5,2,3))
model = MyModel()
print(model)
model2 = My2Model()
print(model2)

3
MyModel(
  (hidden_layers): Sequential(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=20, out_features=5, bias=True)
    (4): ReLU()
  )
)
My2Model(
  (fc1): Linear(in_features=10, out_features=20, bias=True)
  (relu1): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=20, out_features=5, bias=True)
  (relu2): ReLU()
)
